# Multi-Agent Validation with RAG vs Graph-RAG: Hallucination Detection

This notebook demonstrates how multi-agent systems combined with Graph-RAG reduce hallucinations compared to single-agent RAG approaches.

## Research Background

Based on recent research:
- [Teaming LLMs to Detect and Mitigate Hallucinations](https://arxiv.org/pdf/2510.19507)
- [RAG-KG-IL: Multi-Agent Hybrid Framework for Reducing Hallucinations](https://arxiv.org/pdf/2503.13514)
- [MetaRAG: Metamorphic Testing for Hallucination Detection](https://arxiv.org/pdf/2509.09360)
- [Synergistic Integration in Multi-Agent RAG Systems](https://arxiv.org/html/2511.21729v1)

## Key Hypothesis

Multi-agent validation combined with Graph-RAG provides superior hallucination detection through:
1. **Agent specialization** - Executor, Validator, and Critic roles
2. **Cross-validation** - Multiple agents verify each other's outputs
3. **Structured data** - Graph-RAG provides verifiable facts
4. **Explicit reasoning** - Agents must justify their decisions

## Setup

In [ ]:
import sys
import os
import warnings
import logging

# Suppress OpenTelemetry warnings completely
warnings.filterwarnings('ignore')
os.environ['OTEL_SDK_DISABLED'] = 'true'
logging.getLogger('opentelemetry').setLevel(logging.CRITICAL)

# Redirect stderr to suppress context warnings
import io
import contextlib

@contextlib.contextmanager
def suppress_stderr():
    old_stderr = sys.stderr
    sys.stderr = io.StringIO()
    try:
        yield
    finally:
        sys.stderr = old_stderr

# Add path for basic booking tools
sys.path.insert(0, '../semantic-tools-demo')

from strands import Agent
from strands.multiagent import Swarm
from tools import search_hotels, book_hotel, get_booking

print("✅ Basic tools loaded")

# Try to load Graph-RAG tools (optional)
GRAPH_RAG_AVAILABLE = False
try:
    os.environ['NEO4J_URI'] = os.getenv('NEO4J_URI', 'neo4j://127.0.0.1:7687')
    os.environ['NEO4J_USER'] = os.getenv('NEO4J_USER', 'neo4j')
    os.environ['NEO4J_PASSWORD'] = os.getenv('NEO4J_PASSWORD', 'Eli12345678')
    
    sys.path.insert(0, '../hotel-rag-demo/tools')
    from graph_tool import search_hotels_by_country, get_top_rated_hotels, compare_countries
    GRAPH_RAG_AVAILABLE = True
    print("✅ Graph-RAG tools loaded")
except Exception as e:
    print(f"⚠️  Graph-RAG tools not available: {e}")
    print("   Notebook will run with basic tools only")

## Ground Truth: Hotel Booking Database

Our simulated booking system has:
- **3 hotels**: grand_hotel ($200), budget_inn ($80), luxury_resort ($500, unavailable)
- **Booking operations**: search, book, get_booking
- **Validation criteria**: Hotel exists, is available, correct pricing

This ground truth allows us to detect:
1. **Tool misuse** - Using wrong tool for the task
2. **Fabricated success** - Claiming booking succeeded when it failed
3. **Incorrect information** - Wrong prices, availability, or hotel names

In [ ]:
# Ground truth for validation
GROUND_TRUTH_HOTELS = {
    "grand_hotel": {"price": 200, "available": True, "max_guests": 4},
    "budget_inn": {"price": 80, "available": True, "max_guests": 2},
    "luxury_resort": {"price": 500, "available": False, "max_guests": 6}
}

print("📊 Ground Truth - Available Hotels:")
for hotel, info in GROUND_TRUTH_HOTELS.items():
    status = "✅ Available" if info["available"] else "❌ Unavailable"
    print(f"   {hotel}: ${info['price']}/night, max {info['max_guests']} guests - {status}")

## Architecture Comparison

### Single Agent with RAG (Baseline)
```
User Query → Agent + RAG Tools → Response
```
**Risk**: Agent can hallucinate without detection

### Multi-Agent with Basic Tools
```
User Query → Executor → Validator → Critic → Response
              (tools)    (checks)   (approves)
```
**Improvement**: Cross-validation catches some hallucinations

### Multi-Agent with Graph-RAG (Best)
```
User Query → Executor + Graph-RAG → Validator → Critic → Response
              (structured data)      (verifies)  (approves)
```
**Advantage**: Structured data + multi-agent validation = maximum accuracy

---
## Test 1: Single Agent with RAG (Baseline)

In [ ]:
MODEL = "us.anthropic.claude-3-haiku-20240307-v1:0"

# Single agent with booking tools only
single_agent = Agent(
    name="single_agent",
    system_prompt="You are a hotel booking assistant. Use tools to complete requests.",
    tools=[search_hotels, book_hotel, get_booking],
    model=MODEL
)

print("✅ Single agent created")

In [ ]:
query = "Book grand_hotel for John for 2 nights"

print("="*70)
print("TEST 1: Single Agent (Baseline)")
print("="*70)
print(f"Query: {query}\n")

result = single_agent(query)
response_text = result.message['content'][0]['text'] if result.message['content'] else 'No response'
print(f"\n📝 Response:\n{response_text[:200]}...")
print(f"\n🔧 Tools used: {list(result.metrics.tool_metrics.keys())}")


---
## Test 2: Multi-Agent Validation (No RAG)

In [ ]:
# Executor: performs booking actions
executor = Agent(
    name="executor",
    system_prompt="""You are an executor agent. Use tools to complete booking requests.
IMPORTANT: After EVERY action, call handoff_to_agent to pass to 'validator'.
Never complete without handing off first.""",
    tools=[search_hotels, book_hotel, get_booking],
    model=MODEL
)

# Validator: checks for hallucinations
validator = Agent(
    name="validator",
    system_prompt="""You validate booking responses. Check:
- Was the correct tool used?
- Is the response accurate and complete?
- Are there any inconsistencies?
Say VALID or HALLUCINATION with specific reasons.
Then call handoff_to_agent to pass to 'critic'.""",
    model=MODEL
)

# Critic: final approval
critic = Agent(
    name="critic",
    system_prompt="""You are the final critic. Review the entire conversation and say APPROVED or REJECTED.
Provide clear reasoning. You are the last agent - do NOT hand off.""",
    model=MODEL
)

swarm_basic = Swarm([executor, validator, critic], entry_point=executor, max_handoffs=5)
print("✅ Multi-agent swarm created (basic tools)")

In [ ]:
print("="*70)
print("TEST 2: Multi-Agent Validation (Basic Tools)")
print("="*70)
print(f"Query: {query}\n")

with suppress_stderr():
    result = swarm_basic(query)

print(f"\n🔄 Agent Flow: {' → '.join([n.node_id for n in result.node_history])}")
print(f"📊 Status: {result.status}")

# Get final response
final_node = result.node_history[-1].node_id
final_result = result.results[final_node].result
if final_result.message['content']:
    final_text = final_result.message['content'][0]['text']
    print(f"\n📝 Final Response:\n{final_text[:200]}...")


---
## Test 3: Multi-Agent with Graph-RAG Integration

Now we add Graph-RAG tools to provide structured hotel data for validation.

**Note**: This test requires Neo4j running locally. If not available, it will be skipped.

In [ ]:
if GRAPH_RAG_AVAILABLE:
    # Executor with Graph-RAG tools
    executor_graph = Agent(
        name="executor",
        system_prompt="""You are an executor agent with access to both booking tools and hotel database.
Use graph tools (search_hotels_by_country, get_top_rated_hotels) to verify hotel information.
Use booking tools (search_hotels, book_hotel) to complete bookings.
After actions, call handoff_to_agent to pass to 'validator'.""",
        tools=[search_hotels, book_hotel, get_booking, search_hotels_by_country, get_top_rated_hotels, compare_countries],
        model=MODEL
    )
    
    # Validator with Graph-RAG access
    validator_graph = Agent(
        name="validator",
        system_prompt="""You validate responses using the hotel database.
Use graph tools to verify:
- Hotel exists in database
- Ratings and information are accurate
- Country/location is correct
Say VALID or HALLUCINATION with evidence from database.
Then call handoff_to_agent to pass to 'critic'.""",
        tools=[search_hotels_by_country, get_top_rated_hotels, compare_countries],
        model=MODEL
    )
    
    swarm_graph = Swarm([executor_graph, validator_graph, critic], entry_point=executor_graph, max_handoffs=6)
    print("✅ Multi-agent swarm created (with Graph-RAG)")
else:
    print("⚠️  Skipping Graph-RAG test - Neo4j not available")
    print("   To enable: Install Neo4j and set NEO4J_PASSWORD environment variable")

In [ ]:
if GRAPH_RAG_AVAILABLE:
    print("="*70)
    print("TEST 3: Multi-Agent with Graph-RAG")
    print("="*70)
    print(f"Query: {query}\n")
    
    with suppress_stderr():
        result = swarm_graph(query)
    
    print(f"\n🔄 Agent Flow: {' → '.join([n.node_id for n in result.node_history])}")
    print(f"📊 Status: {result.status}")
    
    # Get final response
    final_node = result.node_history[-1].node_id
    final_result = result.results[final_node].result
    if final_result.message['content']:
        final_text = final_result.message['content'][0]['text']
        print(f"\n📝 Final Response:\n{final_text[:200]}...")
else:
    print("⚠️  Skipping - Graph-RAG not available")

---
## Test 4: Hallucination Detection - Invalid Hotel

Test how each approach handles requests for non-existent hotels.

In [ ]:
invalid_query = "Book the_ritz_paris for Sarah for 3 nights"

print("="*70)
print("TEST 4: Hallucination Detection (Invalid Hotel)")
print("="*70)
print(f"⚠️  'the_ritz_paris' does NOT exist in our system!")
print(f"Query: {invalid_query}\n")

In [ ]:
print("--- Single Agent ---")
result_single = single_agent(invalid_query)
response_text = result_single.message['content'][0]['text'] if result_single.message['content'] else 'No response'
print(f"Response: {response_text[:150]}...\n")

In [ ]:
print("--- Multi-Agent (Basic) ---")
with suppress_stderr():
    result_basic = swarm_basic(invalid_query)

print(f"Status: {result_basic.status}")
print(f"Flow: {' → '.join([n.node_id for n in result_basic.node_history])}")

final_node = result_basic.node_history[-1].node_id
final_result = result_basic.results[final_node].result
if final_result.message['content']:
    final_text = final_result.message['content'][0]['text']
    print(f"Response: {final_text[:150]}...\n")

In [ ]:
if GRAPH_RAG_AVAILABLE:
    print("--- Multi-Agent (Graph-RAG) ---")
    with suppress_stderr():
        result_graph = swarm_graph(invalid_query)
    
    print(f"Status: {result_graph.status}")
    print(f"Flow: {' → '.join([n.node_id for n in result_graph.node_history])}")
    
    final_node = result_graph.node_history[-1].node_id
    final_result = result_graph.results[final_node].result
    if final_result.message['content']:
        final_text = final_result.message['content'][0]['text']
        print(f"Response: {final_text[:150]}...")
else:
    print("⚠️  Skipping - Graph-RAG not available")

---
## Test 5: Complex Query - Hotel Recommendation with Booking

Test multi-step reasoning: find best hotel in a country, then book it.

In [ ]:
complex_query = "Find the best rated hotel in Spain and book it for Maria for 1 night"

print("="*70)
print("TEST 5: Complex Query (Recommendation + Booking)")
print("="*70)
print(f"Query: {complex_query}\n")
print("This requires:")
print("1. Query Graph-RAG for best hotel in Spain")
print("2. Extract hotel name from results")
print("3. Book that specific hotel")
print("4. Validate the entire flow\n")

In [ ]:
print("--- Single Agent (will likely fail) ---")
result_single = single_agent(complex_query)
response_text = result_single.message['content'][0]['text'] if result_single.message['content'] else 'No response'
print(f"Response: {response_text[:150]}...\n")

In [ ]:
if GRAPH_RAG_AVAILABLE:
    print("--- Multi-Agent with Graph-RAG ---")
    with suppress_stderr():
        result_graph = swarm_graph(complex_query)
    
    print(f"Status: {result_graph.status}")
    print(f"Flow: {' → '.join([n.node_id for n in result_graph.node_history])}")
    
    final_node = result_graph.node_history[-1].node_id
    final_result = result_graph.results[final_node].result
    if final_result.message['content']:
        final_text = final_result.message['content'][0]['text']
        print(f"Response: {final_text[:150]}...")
else:
    print("⚠️  Skipping - Graph-RAG not available")

---
## Test 6: Out-of-Domain Query

Test how agents handle queries about data that doesn't exist in either system.

In [ ]:
ood_query = "Book a hotel in Switzerland with mountain views for 2 nights"

print("="*70)
print("TEST 6: Out-of-Domain Query")
print("="*70)
print(f"⚠️  Switzerland is NOT in our database!")
print(f"Query: {ood_query}\n")

In [ ]:
print("--- Single Agent ---")
result_single = single_agent(ood_query)
response_text = result_single.message['content'][0]['text'] if result_single.message['content'] else 'No response'
print(f"Response: {response_text[:150]}...\n")

In [ ]:
if GRAPH_RAG_AVAILABLE:
    print("--- Multi-Agent with Graph-RAG ---")
    with suppress_stderr():
        result_graph = swarm_graph(ood_query)
    
    print(f"Status: {result_graph.status}")
    print(f"Flow: {' → '.join([n.node_id for n in result_graph.node_history])}")
    
    final_node = result_graph.node_history[-1].node_id
    final_result = result_graph.results[final_node].result
    if final_result.message['content']:
        final_text = final_result.message['content'][0]['text']
        print(f"Response: {final_text[:150]}...")
else:
    print("⚠️  Skipping - Graph-RAG not available")

---
## Summary & Conclusions

### Hallucination Detection Results

| Test | Single Agent | Multi-Agent (Basic) | Multi-Agent + Graph-RAG |
|------|--------------|---------------------|-------------------------|
| Simple booking | ✅ Works | ✅ Validated | ✅ Validated + Verified |
| Invalid hotel | ⚠️ May fabricate | ✅ Catches error | ✅ Explicit verification |
| Complex query | ❌ Limited tools | ⚠️ Partial | ✅ Full reasoning chain |
| Out-of-domain | ⚠️ May suggest alternatives | ✅ Validates failure | ✅ Database confirms absence |

### Key Findings

#### 1. Multi-Agent Validation Effectiveness
- **Executor-Validator-Critic** pattern catches tool misuse and fabricated responses
- Shared context allows validators to check executor's work
- Explicit handoffs create audit trail

#### 2. Graph-RAG Advantages
- **Structured verification**: Database queries return definitive answers
- **Multi-step reasoning**: Can query hotel DB, then use results for booking
- **Explicit failures**: Missing data returns empty results, not hallucinations

#### 3. Synergistic Effect
Multi-agent + Graph-RAG provides:
- **Cross-validation**: Agents verify each other using structured data
- **Evidence-based reasoning**: Validators cite database results
- **Transparent failures**: System explicitly states when data is unavailable

### Limitations

- Multi-agent systems add latency (multiple LLM calls)
- Graph-RAG requires upfront schema design
- Handoff logic can fail if agents don't follow instructions
- Both systems limited by tool design (e.g., booking tools vs. hotel database mismatch)

### Best Practices

1. **Use multi-agent for high-stakes operations** (bookings, transactions)
2. **Combine with Graph-RAG when structured data exists** (hotels, products, inventory)
3. **Design clear agent roles** (executor, validator, critic)
4. **Provide validators with verification tools** (database access)
5. **Set max handoffs** to prevent infinite loops

### References

- [Teaming LLMs to Detect and Mitigate Hallucinations](https://arxiv.org/pdf/2510.19507)
- [RAG-KG-IL: Multi-Agent Hybrid Framework](https://arxiv.org/pdf/2503.13514)
- [MetaRAG: Metamorphic Testing for Hallucination Detection](https://arxiv.org/pdf/2509.09360)
- [Synergistic Integration in Multi-Agent RAG Systems](https://arxiv.org/html/2511.21729v1)
- [Knowledge-Graph Based LLM Hallucination Evaluation](https://arxiv.org/html/2407.10793v1)